# Writing Your First Agent Test

So far we tested our agent by doing "vide checks": we run it, look at the output, and tweak until we're satisfied.

In this lesson we'll make it more reliable by adding proper tests.

In the previous lesson, we created our Python project. We implemented a class with tools and created our agent.

That's main.py we have so far:

In [ ]:
import search_agent
import asyncio

agent = search_agent.create_agent()
agent_callback = search_agent.NamedCallback(agent)


async def run_agent(user_prompt: str):

    results = await agent.run(
        user_prompt=user_prompt,
        event_stream_handler=agent_callback
    )

    return results


result = asyncio.run(run_agent("What is LLM evaluation?"))

print(result.output)


## Adding Basic Tool Cal Verification

We want the agent to invoke the search tool. Let's add a check to ensure that it happens.

First, we write a function that extract tool calls from messages:



In [ ]:
import json
import asyncio
from dataclasses import dataclass

@dataclass
class ToolCall:
    name: str
    args: dict


def get_tool_calls(result) -> list[ToolCall]:
    calls = []

    for m in result.new_messages():
        for p in m.parts:
            kind = p.part_kind
            if kind == 'tool-call': 
                call = ToolCall(
                    name=p.tool_name,
                    args=json.loads(p.args)
                )
                calls.append(call)

    return calls

Run the agent:

In [ ]:
result = asyncio.run(run_agent("What is LLM evaluation?"))

print(result.output)

Now we can do the test:

In [ ]:
tool_calls = get_tool_calls(result)
assert len(tool_calls) > 0, "No tool calls found"

print("TOOL CALLS:", tool_calls)


Here we use assert to test that at least one tool call was made, ensuring our agent is actively using its search functionality.

You can run this to verify it works (or it doesn't work).

However, this approach isn't how we should write tests.

## Setting Up Proper Testing Infrastructure

We want to split the production code - in our case, main.py, - from tests. Our main.py shouldn't be a testing ground.

Let's take the testing logic out and put it inside a proper test.

We'll use pytest for that - a Python framework for testing.

Install it:

In [ ]:
!uv add --dev pytest

Note that we add it as a dev dependency. Later when others install our project, they'll use uv sync --dev to include these dependencies too (if needed).

Run pytest to see the current state:

In [ ]:
!uv run pytest

`Output: collected 0 items`

Let's create our test structure.

Create tests/ and tests/__init__.py (empty file).

In [ ]:
!mkdir tests
!touch tests/__init__.py

Next, create `conftest.py` in the tests directory.

This file contains shared configuration and fixtures (data) for all tests. It's always executed by pytest before everything else.

We use it to ensure our project modules can be imported properly:

In [ ]:
import sys
from pathlib import Path

# Ensure the project root (parent of tests/) is on sys.path so tests can import project modules
project_root = Path(__file__).resolve().parents[1]
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))


If we don't do it, we won't be able to import our code properly.

We wouldn't need it if we kept all our code in a separate folder (e.g. doc_agent) but we will keep things simpler in this lesson.

Now, create `utils.py` (`test/utils.py`) in the tests directory. We separate utility functions into their own module to keep tests clean and easier to read:

In [ ]:
import json

from dataclasses import dataclass
from typing import List


@dataclass
class ToolCall:
    name: str
    args: dict


def get_tool_calls(result) -> List[ToolCall]:
    """Extract tool-call parts from an agent result and return them as ToolCall objects."""
    calls: List[ToolCall] = []

    for m in result.new_messages():
        for p in m.parts:
            kind = p.part_kind
            if kind == 'tool-call':
                call = ToolCall(
                    name=p.tool_name,
                    args=json.loads(p.args)
                )
                calls.append(call)

    return calls


## Refactoring Main Application

Now remove the testing code from main.py. We can also add a sync version for easier testing:

In [ ]:
import search_agent
import asyncio


agent = search_agent.create_agent()
agent_callback = search_agent.NamedCallback(agent)


async def run_agent(user_prompt: str):
    results = await agent.run(
        user_prompt=user_prompt,
        event_stream_handler=agent_callback
    )

    return results


def run_agent_sync(user_prompt: str):
    return asyncio.run(run_agent(user_prompt))


def main():
    result = run_agent_sync("LLM as a Judge")
    print(result.output)


if __name__ == '__main__':
    main()


## Creating the First Test


Now create a test in `test_agent.py`:

In [ ]:
from main import run_agent_sync
from tests.utils import get_tool_calls


def test_agent_tool_calls_present():
    result = run_agent_sync("LLM as a Judge")
    print(result.output)

    tool_calls = get_tool_calls(result)
    assert len(tool_calls) > 0, "No tool calls found"


We use the `get_tool_calls` function from `utils.py` here.

## Running Tests

Let's run this test now:

In [ ]:
!uv run pytest

To save us some time, we can also create a `Makefile` so we don't need to type the full command every time:

In [ ]:
test:
	uv run pytest

Make sure to use tabs, not spaces for indentation in Makefiles. This is a Makefile requirement that can cause errors if not followed.

Add `.PHONY: test` at the top of the `Makefile` to indicate that "test" is not a file target but a command. This prevents conflicts if someone creates a file named "test":

In [ ]:
.PHONY: test

test:
	uv run pytest


Execute the test with the command:

```bash
make test
```

## Fixing Warnings

You might see this warning:

```
DeprecationWarning: Specifying a model name without a provider prefix is deprecated. Instead of 'gpt-4o-mini', use 'openai:gpt-4o-mini'.
```

Let's fix it.

In [ ]:
def create_agent():
    tools = search_tools.prepare_search_tools()

    return Agent(
        name="search",
        instructions=search_instructions,
        tools=[tools.search],
        model="openai:gpt-4o-mini",
    )

## Viewing Test Output

By default, pytest only shows output from failing tests. If you want to see all test output, run with -s:

In [ ]:
!uv run pytest -s

For now, let's add this flag. We can remove it later once our tests are stable.

## Testing Multiple Tool Calls

Now let's examine our test. If it's passing, that's good!

Let's make it more specific by requiring at least 3 searches:

In [ ]:
def test_agent_makes_3_calls():
    result = run_agent_sync("What is LLM evaluation?")

    tool_calls = get_tool_calls(result)
    assert len(tool_calls) >= 3, "Less than 3 tool calls found"


This test will likely fail initially. Let's update our agent instructions to ensure multiple searches:

In [ ]:
search_instructions = """
You're a helpful assistant that can answer questions by searching the 
documentation.

Make at least 3 searches. 
"""

If this still doesn't work reliably, we need a more explicit approach. Here's a ChatGPT-improved prompt:

In [ ]:
search_instructions = """
You are a helpful assistant that answers questions by searching the documentation.
For every user query, you must perform at least 3 separate searches to gather sufficient context and confirm accuracy.
Each search should explore a different angle or keyword variation of the user's request.
After performing all searches, synthesize the findings into a single, clear, and accurate response.
"""


Now the test should pass, showing output like:

```
TOOL CALL (search): search({"query": "LLM evaluation definition"})
TOOL CALL (search): search({"query": "how to evaluate LLM models"})
TOOL CALL (search): search({"query": "LLM performance metrics"})
```

## Testing Citations and References

Let's add another test for proper citation format. We want the agent to include references in a specific format.

Update the instructions to require proper citations:

In [ ]:
search_instructions = """
You are a helpful assistant that answers questions by searching the documentation.

Requirements:

1. For every user query, you must perform at least 3 separate searches to gather enough context and verify accuracy.  
2. Each search should use a different angle, phrasing, or keyword variation of the user's query.  
3. The search results return filenames (e.g., examples/GitHub_actions.mdx).  
   When citing sources, convert filenames into full GitHub URLs using the following pattern:  
   https://github.com/evidentlyai/docs/blob/main/<filename>  
   Example:  
   examples/GitHub_actions.mdx → https://github.com/evidentlyai/docs/blob/main/examples/GitHub_actions.mdx  
4. After performing all searches, write a concise, accurate answer that synthesizes the findings.  
5. At the end of your response, include a "References" section listing all the sources you used, one per line, in the format:

## References

- [Title or Filename](https://github.com/evidentlyai/docs/blob/main/path/to/file.mdx)
- ...
"""


Then test that the agent follows the citation format:

In [ ]:
assert "## References" in output
assert "(https://github.com/evidentlyai/docs/blob/main/" in output

## Testing Code Examples

Let's create another test (test_agent_code) for code-related questions:

"How do I implement LLM as a Judge eval?"

We expect to have some Python code blocks there. We can explicitly test it:

In [ ]:
assert "```python" in output

We have now two tests. If we want to run this specific test, use this command:

In [ ]:
!uv run pytest tests/test_agent.py::test_agent_code -s

## Benefits of Agent Testing

It's important to test your agents. We can tweak some parameters while improving one thing, and accidentally break something else.

But with tests, if you change prompts or configuration later, tests ensure your requirements are still satisfied. Manual "vibe checking" can miss subtle regressions.

So when you modify one aspect of your agent, tests verify that other functionality still works correctly. Without tests, you might forget to check all the different capabilities.

We run tests automatically and ongoingly, so we can catch issues before they reach production. This is especially important for agents where behavior can be unpredictable.
